## Текстовый анализ URL в задаче lookalike

In [1]:
#Config
from pyspark import SparkConf, SparkContext, HiveContext
import re
import numpy as np
import pandas as pd
import datetime
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.evaluation import BinaryClassificationMetrics

hive_config_query = '''
set hive.vectorized.execution.enabled=true;
set hive.vectorized.execution.reduce.enabled = true;
set mapreduce.map.memory.mb=4096;
set mapreduce.map.child.java.opts=-Xmx4g;
set mapreduce.task.io.sort.mb=1024;
set mapreduce.reduce.child.java.opts=-Xmx4g;
set mapreduce.reduce.memory.mb=7000;
set mapreduce.reduce.shuffle.input.buffer.percent=0.5;
set mapreduce.input.fileinputformat.split.minsize=536870912;
set mapreduce.input.fileinputformat.split.maxsize=1073741824;
set hive.optimize.ppd=true;
set hive.merge.smallfiles.avgsize=536870912;
set hive.merge.mapredfiles=true;
set hive.merge.mapfiles=true;
set hive.hadoop.supports.splittable.combineinputformat=true;
set hive.exec.reducers.bytes.per.reducer=536870912;
set hive.exec.parallel=true;
set hive.exec.max.created.files=10000000;
set hive.exec.compress.output=true;
set hive.exec.dynamic.partition.mode=nonstrict;
set hive.exec.max.dynamic.partitions=1000000;
set hive.exec.max.dynamic.partitions.pernode=100000;
set io.seqfile.compression.type=BLOCK;
set mapreduce.map.failures.maxpercent=5;
'''
try:
    sc.stop()
except:
    pass

conf = SparkConf().set("spark.executor.instances", 32).set("spark.driver.maxResultSize", "15g").set('spark.driver.memory','15g').set("spark.executor.memory", '4g')
sc = SparkContext(conf=conf)
hc = HiveContext(sc)

for q in hive_config_query.split(';'):
    try:
        hc.sql(q)
    except:
        pass


In [2]:
#Constants
train_date = datetime.date(2016,11,8)
n = 3

test_date = train_date + datetime.timedelta(days = 7)

In [3]:
# Hive queries

train_sample_query = '''

CREATE FUNCTION md5 as 'onemd5.Md5';

create table if not exists user_kposminin.url_text_#ind as
select 
  v.id,
  max(if(u.id is Null,0,1)) as label,
  max(nvl(u.first_day,0)) as first_day,
  split(concat_ws(' ',collect_list(url)),'[ /\\-=_\\?\\.]') as up_bow,
  concat_ws(' ',collect_list(url)) as up
from prod_raw_liveinternet.access_log v
  left join(
    select distinct id, if(ymd = '#ymd1',1,0) as first_day
    from prod_features_liveinternet.user_action
    where action_type = 'tinkoff_platinum_approved_application'
      and ymd between '#ymd1' and '#ymd3'
  ) u on u.id = v.id
where
  v.ymd = '#ymd0' and 
  (substr(md5(v.id),1,2) = '00' or not u.id is Null)
group by 
  v.id
;
select * from user_kposminin.url_text_#ind

'''.replace('#ymd0',str(train_date)) \
   .replace('#ymd1',str(train_date + datetime.timedelta(days = 1))) \
   .replace('#ymd3',str(train_date + datetime.timedelta(days = 3))) \
   .replace('#ind',str(train_date).replace('-',''))

    
    
test_sample_query = '''
create table if not exists user_kposminin.url_text_#ind as
select 
  v.id,
  max(if(u.id is Null,0,1)) as label,
  max(nvl(u.first_day,0)) as first_day,
  split(concat_ws(' ',collect_list(url)),'[ /\\-=_\\?\\.]') as up_bow,
  concat_ws(' ',collect_list(url)) as up
from prod_raw_liveinternet.access_log v
  left join(
    select distinct id, if(ymd = '#ymd1',1,0) as first_day
    from prod_features_liveinternet.user_action
    where action_type = 'tinkoff_platinum_approved_application'
      and ymd between '#ymd1' and '#ymd3'
  ) u on u.id = v.id
where
  ymd = '#ymd0' and 
  substr(md5(v.id),1,2) = '00' 
group by 
  v.id
;

select * from user_kposminin.url_text_#ind

'''.replace('#ymd0',str(test_date)) \
   .replace('#ymd1',str(test_date + datetime.timedelta(days = 1))) \
   .replace('#ymd3',str(test_date + datetime.timedelta(days = 3))) \
   .replace('#ind',str(test_date).replace('-',''))


In [4]:
#abc = list(set(''.join([e[0] for e in hc.sql('select url from prod_raw_liveinternet.access_log v where ymd = "2017-01-10" limit 100000').collect()])))
abc = list(u'abcdefghijklmnopqrstuvwxyz0123456789абвгдеёжзийклмнопрстуфхцчшщъыьэюя %&-./=?_')

def n_gram(s, n):
    '''Returns n-gram list from string s.'''
    return [s[i:i+n] for i in range(len(s) - n + 1)]

def n_gram_index(ngr,abc):
    '''Returns index of n-gram ngr. ngr chars must be from abc list'''
    N = len(abc)
    ind = 0
    for i in range(len(ngr)):
        try:
            ind += (N ** i) * abc.index(ngr[i].lower())
        except ValueError:
            ind += (N ** i) * (N - 1)
    return ind

In [5]:
# Calc in Spark
for q in train_sample_query.split(';')[:-1] + test_sample_query.split(';')[:-1]:
    try:
        hc.sql(q)
    except:
        pass

train = hc.sql(train_sample_query.split(';')[-1]) \
        .rdd \
        .map(lambda r: [r.label, r.first_day, reduce(lambda a,b:a+b,[n_gram(e,n) for e in r.up_bow])]) \
        .map(lambda r: LabeledPoint(float(r[0]),SparseVector(len(abc) ** n,{n_gram_index(e,abc):1.0 for e in list(set(r[2]))})))

test  = hc.sql(test_sample_query.split(';')[-1]) \
        .rdd \
        .map(lambda r: [r.label, r.first_day, reduce(lambda a,b:a+b,[n_gram(e,n) for e in r.up_bow])]) \
        .map(lambda r: LabeledPoint(float(r[0]),SparseVector(len(abc) ** n,{n_gram_index(e,abc):1.0 for e in list(set(r[2]))})))


In [9]:
# Calc local
for q in train_sample_query.split(';')[:-1] + test_sample_query.split(';')[:-1]:
    try:
        hc.sql(q)
    except:
        pass

train = hc.sql(train_sample_query.split(';')[-1]) \
        .collect()

test  = hc.sql(test_sample_query.split(';')[-1]) \
        .collect()


KeyboardInterrupt: 

In [5]:
train = hc.sql(train_sample_query.split(';')[-1]) \
        .collect()
len(train)

KeyboardInterrupt: 

In [ ]:
test  = hc.sql(test_sample_query.split(';')[-1]) \
        .collect()
len(test)

In [ ]:
#LogisticRegression model
modelLR = LogisticRegressionWithSGD.train(train, regType = 'l1',intercept = True,iterations = 10)
modelLR.clearThreshold()

In [ ]:
df_test = test \
                     .map(lambda r: (float(modelLR.predict(r.features)),float(r.label))) \
                     .toDF() \
                     .toPandas()


In [ ]:
1

In [26]:
results = transformed.select(['probability', 'label'])
 
## prepare score-label set
results_collect = results.collect()
results_list = [(float(i[0][0]), 1.0-float(i[1])) for i in results_collect]
scoreAndLabels = sc.parallelize(results_list)
 
metrics = metric(scoreAndLabels)
print("The ROC score is ): ", metrics.areaUnderROC)


lrm.predict(sc.parallelize([[1.0, 0.0], [0.0, 1.0]])).collect()
[1, 0]
lrm.clearThreshold()
lrm.predict([0.0, 1.0])

sparse_data = [
     LabeledPoint(0.0, SparseVector(2, {0: 0.0})),
     LabeledPoint(1.0, SparseVector(2, {1: 1.0})),
...     LabeledPoint(0.0, SparseVector(2, {0: 1.0})),
...     LabeledPoint(1.0, SparseVector(2, {1: 2.0}))
... ]



[3, 4]

In [ ]:
>>> sparse_data = [
...     LabeledPoint(0.0, SparseVector(2, {0: 0.0})),
...     LabeledPoint(1.0, SparseVector(2, {1: 1.0})),
...     LabeledPoint(0.0, SparseVector(2, {0: 1.0})),
...     LabeledPoint(1.0, SparseVector(2, {1: 2.0}))
... ]